In [1]:
from processor.processor import Processor as processor
from database.adatabase import ADatabase
from xgboost import XGBRegressor
from statistics import mean
import math
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
from dotenv import load_dotenv
from asset.stock import Stock
from asset.bond import Bond
from asset.option import Option
import warnings
import pytz
import copy
warnings.simplefilter(action="ignore")
import pickle

In [2]:
market = ADatabase("market")
sec = ADatabase("sec")
market = ADatabase("market")
fred = ADatabase("fred")
db = ADatabase("sapling")

In [3]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",attrs={"id":"constituents"})[0].rename(columns={"Symbol":"ticker"})

In [4]:
fred.connect()
spy = fred.retrieve("sp500")
spy = spy.rename(columns={"value":"spy"})
spy["spy"] = spy["spy"].replace(".",np.nan)
spy.dropna(inplace=True)
spy["spy"] = [float(x) for x in spy["spy"]]
spy["date"] = spy["date"].shift(-5)
spy = processor.column_date_processing(spy)
fred.disconnect()

In [5]:
prices = []
market.connect()

for ticker in tqdm(sp500["ticker"].unique()):
    try:
        price = processor.column_date_processing(market.query("prices",{"ticker":ticker}))
        price.sort_values("date",inplace=True)
        price["y"] = price["adjclose"].shift(-5)
        prices.append(price)
    except Exception as e:
        print(ticker,str(e))
        continue
market.disconnect()

  2%|█████                                                                                                                                                                                                                | 12/503 [00:00<00:04, 113.18it/s]

MMM 'date'
AOS 'date'
AMD 'date'
AFL 'date'
A 'date'
APD 'date'
ARE 'date'
LNT 'date'
GOOGL 'date'
GOOG 'date'
MO 'date'
AMZN 'date'


 10%|█████████████████████▏                                                                                                                                                                                               | 50/503 [00:00<00:04, 112.48it/s]

AXP 'date'
AWK 'date'
APH 'date'
ANSS 'date'
AON 'date'
APA 'date'
AMAT 'date'
APTV 'date'
ANET 'date'
T 'date'
ATO 'date'


 16%|█████████████████████████████████                                                                                                                                                                                    | 78/503 [00:00<00:03, 120.03it/s]

...

TMO 'date'
TSCO 'date'
TT 'date'
TYL 'date'
USB 'date'
UBER 'date'
UDR 'date'
UPS 'date'
URI 'date'
VLO 'date'
VTR 'date'
VLTO 'date'


 96%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍       | 485/503 [00:04<00:00, 109.63it/s]

VZ 'date'
VTRS 'date'
V 'date'
VST 'date'
WRB 'date'
WMT 'date'
WBD 'date'
WM 'date'
WEC 'date'
WFC 'date'
WY 'date'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503/503 [00:04<00:00, 112.41it/s]

WYNN 'date'
XEL 'date'
XYL 'date'
YUM 'date'


In [6]:
training_data = pd.concat(prices).merge(spy[["date","spy"]],how="left",on="date").sort_values("date").reset_index(drop=True)

In [7]:
model = XGBRegressor()
model.fit(training_data[["adjclose","spy"]].iloc[:int(training_data.index.size*0.7)]
          ,training_data[["y"]].iloc[:int(training_data.index.size*0.7)])

XGBoostError: [05:57:37] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\data\data.cc:507: Check failed: valid: Label contains NaN, infinity or a value too large.

In [ ]:
db.cloud_connect()
db.drop("model")
db.store("model",pd.DataFrame([{"model":pickle.dumps(model)}]))
db.disconnect() 